# CS5242

## Load Data

In [1]:
from data import Data

train_data = Data(data_type='train')
test_data = Data(data_type='test')

print("Train data: x:"+ str(train_data.x.shape)+ ' y:'+str(train_data.y.shape))
print("Test data: x:"+ str(test_data.x.shape)+ ' y:'+str(test_data.y.shape))



Loading train data 18662/18662: 100%|█████████████████████████████████████████████████████████████ [ time left: 00:00 ]
Loading test data 6051/6051: 100%|████████████████████████████████████████████████████████████████ [ time left: 00:00 ]


Train data: x:(18662, 1000, 102) y:(18662, 1)
Test data: x:(6051, 1000, 102) y:(6051, 1)


## Model

In [2]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences=True)),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

## Save Checkpoint

In [3]:
import os
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints_lstm_256_temperal'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [5]:
history = model.fit(x=train_data.x, y=train_data.y, validation_split=0.15, epochs=1000,batch_size=64,
                    callbacks=[checkpoint_callback], verbose=2,shuffle=True)

Train on 15862 samples, validate on 2800 samples
Epoch 1/1000
15862/15862 - 51s - loss: 0.3522 - accuracy: 0.8481 - val_loss: 0.3560 - val_accuracy: 0.8525
Epoch 2/1000
15862/15862 - 52s - loss: 0.2615 - accuracy: 0.8747 - val_loss: 0.3211 - val_accuracy: 0.8582
Epoch 3/1000
15862/15862 - 53s - loss: 0.2251 - accuracy: 0.8894 - val_loss: 0.3042 - val_accuracy: 0.8789
Epoch 4/1000
15862/15862 - 53s - loss: 0.1929 - accuracy: 0.9035 - val_loss: 0.2700 - val_accuracy: 0.9118
Epoch 5/1000
15862/15862 - 53s - loss: 0.1777 - accuracy: 0.9195 - val_loss: 0.2601 - val_accuracy: 0.8996
Epoch 6/1000
15862/15862 - 53s - loss: 0.1661 - accuracy: 0.9257 - val_loss: 0.3061 - val_accuracy: 0.9111
Epoch 7/1000
15862/15862 - 53s - loss: 0.1508 - accuracy: 0.9350 - val_loss: 0.2830 - val_accuracy: 0.9179
Epoch 8/1000
15862/15862 - 53s - loss: 0.1470 - accuracy: 0.9399 - val_loss: 0.3435 - val_accuracy: 0.9182
Epoch 9/1000
15862/15862 - 51s - loss: 0.1381 - accuracy: 0.9406 - val_loss: 0.3056 - val_accur

UnknownError: Failed to WriteFile: ./training_checkpoints_lstm_256_temperal\ckpt_394_temp_9ce62a8f95f54dd5b346dce796551aad/part-00001-of-00002.data-00000-of-00001.tempstate13248272306088401782 : There is not enough space on the disk.
; Unknown error [Op:SaveV2]

## Plot Training History

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

## Model Summary

In [ ]:
model.summary()

## Restore Weights

In [4]:
checkpoint_dir = './training_checkpoints_lstm_256_temperal'
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

In [5]:
test_predict = model.predict(test_data.x)
print(test_predict.shape)

(6051, 1)




## Save Prediction to CSV

In [6]:
import csv

output_file = 'result_lstm_256_temperal.csv'
with open(output_file, 'wt', newline='', encoding='utf-8') as output_file:
    csv_writer = csv.writer(output_file)
    csv_writer.writerow(('Id', 'Predicted'))
    for id, predict in enumerate(test_predict):
        csv_writer.writerow((id, predict[0]))